# **Modelling and Evaluating**

## Objectives

* Answer business requirement 2: The client is interested in predicting if a cherry leaf is healthy or contains powdery mildew.

## Inputs

* inputs/cherry_tree_dataset/cherry-leaves/test
* inputs/cherry_tree_dataset/cherry-leaves/train
* inputs/cherry_tree_dataset/cherry-leaves/validation

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* No comments 



---

## Import desired libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
sns.set_style("white")
from matplotlib.image import imread

In [2]:
%pip install -r /workspace/mildew-in-cherry-trees/requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Set the working directory

In [3]:
cwd= os.getcwd()

In [4]:
os.chdir('/workspace/mildew-in-cherry-trees')
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
work_dir = os.getcwd()
work_dir

'/workspace/mildew-in-cherry-trees'

### Set input directory

Set the train, test and validation paths.

In [ ]:
my_data_dir = 'inputs/cherry_tree_dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

### Set output directory

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### Set label names

In [ ]:
# Set the labels
labels = os.listdir(train_path)
print('Label for the images are', labels)

---

# Data Visualization

### Data cleaning

Check and remove any non image files

---

# Conclusion and next steps

### Conclusion

* The outcome of this notebook was to obtain the data from Kaggle, check for any non image files, remove them if necessary and split the data between train, test and validation.

* The data set contained 4208 images of leaves. Evenly split in the middle, healthy and not healthy.

* The data was then split into test, train and validation. 70% train, 10% validation and 20% test.

### The next steps

* The next notebook will contain the data visulaization part. Focussing on the first business requirement - The client is interested in being able to visually differentiate a cherry leaf that is healthy from one that contains powdery mildew.